In [12]:
import sys; sys.path.append("..")
import warnings; warnings.filterwarnings("ignore")
from dotenv import load_dotenv; load_dotenv()
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import json
import yaml

In [55]:
from rag.embed import EmbedChunks
from rag.generate import get_sources_and_context, QueryAgent, ComparisonAgent
from rag.prompts import DOCUMENT_QA_SYSTEM_PROMPT, DOCUMENT_QA_USER_PROMPT_TEMPLATE, FINAL_RESPONSE_SYSTEM_PROMPT_TEMPLATE
from rag.config import MAX_CONTEXT_LENGTHS

In [16]:
with open('../config.YAML') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [36]:
with open('../data/Constituciones.json') as f:
    constituciones = json.load(f)

In [17]:
embedder = EmbedChunks(config["embedding_model"])

In [7]:
get_sources_and_context(query="El Banco Central", embedding_model=embedder.embedding_model, num_chunks=3, id_constitucion=1)

([201, 202, 125],
 ['Capítulo XIII: BANCO CENTRAL, Artículo 108',
  'Capítulo XIII: BANCO CENTRAL, Artículo 109',
  'Capítulo V: CONGRESO NACIONAL, Artículo 63'],
 [{'Source 1:\nCapítulo XIII: BANCO CENTRAL\nArtículo 108\n\nExistirá un organismo autónomo, con patrimonio propio, de carácter técnico, denominado Banco Central, cuya composición, organización, funciones y atribuciones determinará una ley orgánica constitucional.\n'},
  {'Source 2:\nCapítulo XIII: BANCO CENTRAL\nArtículo 109\n\nEl Banco Central sólo podrá efectuar operaciones con instituciones financieras, sean públicas o privadas. De manera alguna podrá otorgar a ellas su garantía, ni adquirir documentos emitidos por el Estado, sus organismos o empresas.\nSin perjuicio de lo anterior, en situaciones excepcionales y transitorias, en las que así lo requiera la preservación del normal funcionamiento de los pagos internos y externos, el Banco Central podrá comprar durante un período determinado y vender, en el mercado secundari

In [8]:
get_sources_and_context(query="El Banco Central", embedding_model=embedder.embedding_model, num_chunks=3, id_constitucion=2)

([731, 733, 732],
 ['Capítulo X: Órganos Autónomos Constitucionales, Artículo 357',
  'Capítulo X: Órganos Autónomos Constitucionales, Artículo 359',
  'Capítulo X: Órganos Autónomos Constitucionales, Artículo 358'],
 [{'Source 1:\nCapítulo X: Órganos Autónomos Constitucionales\nArtículo 357\n\n1. El Banco Central es un órgano autónomo con personalidad jurídica y patrimonio propio, de carácter técnico, encargado de formular y conducir la política monetaria.\n2. La ley regulará su organización, atribuciones y sistemas de control, así como la determinación de instancias de coordinación entre el Banco y el Gobierno.\n'},
  {'Source 2:\nCapítulo X: Órganos Autónomos Constitucionales\nArtículo 359\n\nSon atribuciones del Banco Central la regulación de la cantidad de dinero y de crédito en circulación, la ejecución de operaciones de crédito y cambios internacionales, y la potestad para dictar normas en materia monetaria, crediticia, financiera y de cambios internacionales, y las demás que es

In [58]:
constitucion1_agent = QueryAgent(embedding_model_name=config["embedding_model"],
                                llm=config["chat_model"],
                                temperature=config["temperature"],
                                max_context_length=MAX_CONTEXT_LENGTHS[config["chat_model"]],
                                system_content=DOCUMENT_QA_SYSTEM_PROMPT,
                                assistant_content="",
                                constitucion_id=1)

first_response = constitucion1_agent(query='El Banco Central',
                                    num_chunks=3,
                                    stream=False)['answer']

constitucion2_agent = QueryAgent(embedding_model_name=config["embedding_model"],
                                llm=config["chat_model"],
                                temperature=config["temperature"],
                                max_context_length=MAX_CONTEXT_LENGTHS[config["chat_model"]],
                                system_content=DOCUMENT_QA_SYSTEM_PROMPT,
                                assistant_content="",
                                constitucion_id=2)

second_response = constitucion2_agent(query='El Banco Central',
                                    num_chunks=3,
                                    stream=False)['answer']

In [59]:
final_agent = ComparisonAgent(embedding_model_name=config["embedding_model"],
                        llm=config["chat_model"],
                        temperature=config["temperature"],
                        max_context_length=MAX_CONTEXT_LENGTHS[config["chat_model"]],
                        system_content=FINAL_RESPONSE_SYSTEM_PROMPT_TEMPLATE.render(document1_title=constituciones[str(1)],
                                                                                    document2_title=constituciones[str(2)]),
                        assistant_content="")

In [60]:
final_agent(query='El Banco Central',
            first_response=first_response,
            second_response=second_response,
            stream=False)['answer']

'El Banco Central de Chile es el organismo encargado de velar por la estabilidad y el correcto funcionamiento del sistema financiero del país. Su principal objetivo es mantener la estabilidad de precios y promover el desarrollo económico sostenible. Fue creado en 1925 y su autonomía está consagrada en la Constitución Actual 1980, en el artículo 97.\n\nEl Anteproyecto Expertos 2023 propone mantener la existencia del Banco Central como institución autónoma, pero establece algunas modificaciones en su estructura y funciones. Por ejemplo, se propone fortalecer su independencia política y económica, así como su capacidad para regular y supervisar el sistema financiero. Además, se plantea la posibilidad de ampliar su mandato para incluir objetivos relacionados con el desarrollo social y la protección del medio ambiente.\n\nEn resumen, tanto la Constitución Actual 1980 como el Anteproyecto Expertos 2023 reconocen la importancia del Banco Central como institución autónoma encargada de velar po